In [3]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [4]:
dat = pd.read_csv('goodreads.tsv' , sep='\t') 
print(dat.shape)
dat.head()


(8433, 4)


,title,date,rating,body
0,The Body: A Guide for Occupants,"Oct 11, 2019",5.0,Lovingly presented with humour and kindness an...
1,Resistance Reborn,"Feb 28, 2020",4.0,I read through this book quicker than anticip...
2,The Book of Two Ways,"Jul 13, 2020",4.0,Jodi Picoult does it again in this new novel. ...
3,How to Write One Song: Loving the Things We Cr...,"Feb 21, 2021",4.0,I’m a music freak without a shred of musical a...
4,The Giver of Stars,"Sep 08, 2020",4.0,The Giver of Stars by Jojo Moyes won the Book ...


In [5]:
#Dropping Null Values
df=dat.dropna()
df

,title,date,rating,body
0,The Body: A Guide for Occupants,"Oct 11, 2019",5.0,Lovingly presented with humour and kindness an...
1,Resistance Reborn,"Feb 28, 2020",4.0,I read through this book quicker than anticip...
2,The Book of Two Ways,"Jul 13, 2020",4.0,Jodi Picoult does it again in this new novel. ...
3,How to Write One Song: Loving the Things We Cr...,"Feb 21, 2021",4.0,I’m a music freak without a shred of musical a...
4,The Giver of Stars,"Sep 08, 2020",4.0,The Giver of Stars by Jojo Moyes won the Book ...
...,...,...,...,...
8428,The Giver of Stars,"Feb 12, 2020",5.0,I loved this book! Let me start by saying that...
8429,Minor Feelings: An Asian American Reckoning,"May 21, 2020",4.0,What did i just read?I chose this title becaus...
8430,Trixie and Katya's Guide to Modern Womanhood,"Oct 29, 2020",4.0,**3.45 stars ( if we were using a 10/10 scale ...
8431,To Wake the Giant: A Novel of Pearl Harbor,"May 22, 2020",4.0,"Thanks to Netgalley, Random House and Ballenti..."


In [6]:
#function to represent sentiment -1(negetive);0(neutral);1(positive)
def sentiment(n):
    return 1 if n>=4 else (-1 if n<=2 else 0)
#Applying Sentiment Function 
df['sent'] = df['rating'].apply(sentiment)
df.head()

C:\Users\parde\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,date,rating,body,sent
0,The Body: A Guide for Occupants,"Oct 11, 2019",5.0,Lovingly presented with humour and kindness an...,1
1,Resistance Reborn,"Feb 28, 2020",4.0,I read through this book quicker than anticip...,1
2,The Book of Two Ways,"Jul 13, 2020",4.0,Jodi Picoult does it again in this new novel. ...,1
3,How to Write One Song: Loving the Things We Cr...,"Feb 21, 2021",4.0,I’m a music freak without a shred of musical a...,1
4,The Giver of Stars,"Sep 08, 2020",4.0,The Giver of Stars by Jojo Moyes won the Book ...,1


In [7]:
Sentiment_count=df.groupby('sent').count()
Sentiment_count

,title,date,rating,body
sent,,,,
-1,931,931,931,931
0,1430,1430,1430,1430
1,5778,5778,5778,5778


In [8]:
for i in range(0,len(df)-1):
    if type(df.iloc[i]['body']) != str:
        df.iloc[i]['body'] = str(df.iloc[i]['body'])

df.shape

(8139, 5)

In [9]:
x=df['body']
y=df['sent']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=0)

In [11]:
import re


def clean_text(text):
        
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english",preprocessor=clean_text)
X_train_CV = vectorizer.fit_transform(X_train)
X_test_CV = vectorizer.transform(X_test)
X_train_CV

<6511x28837 sparse matrix of type '<class 'numpy.int64'>'
	with 398863 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_CV)
X_train_tfidf.shape
X_train_tfidf


<6511x28837 sparse matrix of type '<class 'numpy.float64'>'
	with 398863 stored elements in Compressed Sparse Row format>

In [14]:
X_test_tfidf = tfidf_transformer.transform(X_test_CV)
X_test_tfidf

<1628x28837 sparse matrix of type '<class 'numpy.float64'>'
	with 91754 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf = SGDClassifier(loss="log", penalty="l2") #logistic Regression
clf.fit(X_train_CV, y_train)
clf
y_predic  =  clf.predict(X_test_CV)
y_predic

array([ 1,  1,  1, ..., -1,  1,  1], dtype=int64)

In [16]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
clas = classification_report(y_test, y_predic)
clas

'              precision    recall  f1-score   support\n\n          -1       0.48      0.37      0.42       197\n           0       0.37      0.25      0.30       277\n           1       0.81      0.90      0.85      1154\n\n    accuracy                           0.73      1628\n   macro avg       0.55      0.51      0.52      1628\nweighted avg       0.69      0.73      0.70      1628\n'

In [17]:
con = confusion_matrix(y_test, y_predic)
con

array([[  73,   41,   83],
       [  40,   69,  168],
       [  38,   76, 1040]], dtype=int64)

In [18]:
acc = accuracy_score(y_test,y_predic)
acc
print(acc*100)

72.60442260442261


In [19]:
clf = SGDClassifier(loss="hinge", penalty="l2") #svm
clf.fit(X_train_tfidf, y_train)
clf

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
y_pred  =  clf.predict(X_test_tfidf)
y_pred
ac = accuracy_score(y_test,y_pred)
ac
print(acc*100)

72.60442260442261


In [21]:
from sklearn.model_selection import GridSearchCV
params = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "elasticnet", "none"],
}

clf = SGDClassifier(max_iter=100)
grid = GridSearchCV(clf, param_grid=params, cv=10)


grid.fit(X_train_tfidf, y_train)

print(grid.best_params_)

C:\Users\parde\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


{'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1'}


In [36]:
grid_predictions = grid.predict(X_test_tfidf) 

grid_predictions

array([ 1,  1,  1, ..., -1,  1,  1], dtype=int64)

In [38]:
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, grid_predictions)*100))

Accuracy: 74.75


In [39]:
grid_train_predictions = grid.predict(X_train_tfidf) 

print('Accuracy: {:.2f}'.format(accuracy_score(y_train, grid_train_predictions)*100))

Accuracy: 83.34
